# Import Libraries and Datasets

In [3]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [27]:
# Load paths in a dict for train, test, valid folders with wildfire and nowildfire images 

images_path = '/Users/silvanoquarto/Desktop/PROJECTS/Data_Wildfire_Project'
splits = ['train', 'test', 'valid']
labels = ['wildfire', 'nowildfire']

image_paths = {f"{split}_{label}": [] for split in splits for label in labels}

for split in splits:
    for label in labels:
        path = os.path.join(images_path, split, label)
        if os.path.exists(path):  
            image_files = os.listdir(path)
            full_paths = [os.path.join(path, img) for img in image_files]
            image_paths[f"{split}_{label}"].extend(full_paths)
            print(f"- {split.capitalize()} set for {label} uploaded correctly!!")
        else:
            print(f"Warning: {path} does not exist :(")

- Train set for wildfire uploaded correctly!!
- Train set for nowildfire uploaded correctly!!
- Test set for wildfire uploaded correctly!!
- Test set for nowildfire uploaded correctly!!
- Valid set for wildfire uploaded correctly!!
- Valid set for nowildfire uploaded correctly!!


# Data overview about datasets statistcs

In [25]:
# Labels distribution 

for split in splits:
    wildfire_count = len(image_paths[f"{split}_wildfire"])
    nowildfire_count = len(image_paths[f"{split}_nowildfire"])
    total_count = wildfire_count + nowildfire_count
    print("-"*30)
    print(f"{split.capitalize()} set:") 
    print(f"Wildfire: {wildfire_count}")
    print(f"No wildfire: {nowildfire_count}")
    print(f"Total: {total_count}")
    print("-"*30)

train_wildfire: 15750 images
train_nowildfire: 14500 images
test_wildfire: 3480 images
test_nowildfire: 2820 images
valid_wildfire: 3480 images
valid_nowildfire: 2820 images


# Data Visualization

# Image Quality Analysis

# Data Augmentation

# Feature Engineering